# quant-econ Solutions: Markov Asset Pricing

Solutions for http://quant-econ.net/jl/markov_asset.html

In [4]:
include("asset_pricing.jl")  #  Assumes you have this file in the same directory as this notebook

call_option (generic function with 3 methods)

## Exercise 1

In [5]:
# == Define primitives == #
n = 5
P = 0.0125 .* ones(n, n) 
P .+= diagm(0.95 .- 0.0125 .* ones(5))
s = [1.05, 1.025, 1.0, 0.975, 0.95]
gamm = 2.0
bet = 0.94
zet = 1.0
ap = AssetPrices(bet, P, s, gamm)

v = tree_price(ap)
println("Lucas Tree Prices: $v\n")

v_consol = consol_price(ap, 1.0)
println("Consol Bond Prices: $(v_consol)\n")

P_tilde = P .* s'.^(1-gamm)
temp = bet .* (P_tilde * v) + bet * (P_tilde * ones(n))
println("Should be 0: $(temp - v)\n") 

p_s = 150.0
w_bar, w_bars = call_option(ap, zet, p_s, [10, 20, 30]);

Lucas Tree Prices: [12.722217630644252,14.725150017725886,17.571422357262904,21.9357066112197,29.474015777144995]

Consol Bond Prices: [87.5686013853112,109.25108965024711,148.67554548466475,242.55144081989457,753.8710047641986]

Should be 0: [0.0,1.7763568394002505e-15,0.0,0.0,0.0]

